In [ ]:
# --- Celda de Prueba Directa para buscar_coches_bq ---
import pandas as pd
import logging
import traceback
import json # Para imprimir dicts de forma legible

# Ajusta las rutas de importación según tu estructura
from utils.bigquery_tools import buscar_coches_bq 
from config.settings import ( 
    MIN_MAX_RANGES,PENALTY_PUERTAS_BAJAS,PENALTY_LOW_COST_POR_COMODIDAD,
    PENALTY_DEPORTIVIDAD_POR_COMODIDAD,UMBRAL_LOW_COST_PENALIZABLE_SCALED, # Renombrado para claridad
    UMBRAL_DEPORTIVIDAD_PENALIZABLE_SCALED, # Renombrado
    PENALTY_ANTIGUEDAD_MAS_10_ANOS, PENALTY_ANTIGUEDAD_7_A_10_ANOS,
    PENALTY_ANTIGUEDAD_5_A_7_ANOS, BONUS_DISTINTIVO_ECO_CERO_C,
    PENALTY_DISTINTIVO_NA_B, BONUS_OCASION_POR_IMPACTO_AMBIENTAL,
    BONUS_ZBE_DISTINTIVO_FAVORABLE,PENALTY_ZBE_DISTINTIVO_DESFAVORABLE
)

In [ ]:

# Configurar el logging para ver los mensajes de depuración de la función
logging.basicConfig(level=logging.INFO) 
# logging.getLogger("utils.bigquery_tools").setLevel(logging.DEBUG) # Descomenta para logs DEBUG de la función

# --- Configuración de Pandas para Mejor Visualización ---
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None) # Mostrar todas las columnas
pd.set_option('display.width', 2000)    
pd.set_option('display.max_colwidth', 100) 

# --- 1. Escenario de Prueba: "Familiar Práctico con Énfasis en Impacto Ambiental" ---
print("\n--- ESCENARIO 1: FAMILIAR PRÁCTICO - IMPACTO AMBIENTAL ---")
filtros_test_familiar_eco = {
    # Filtros técnicos directos
    "estetica_min": 1.0,
    "premium_min": 1.0,
    "singular_min": 2.0,
    "tipo_mecanica": ['HEVG', 'HEVD', 'PHEVG', 'PHEVD', 'BEV', 'GLP'], # Favoreciendo eco
    "tipo_carroceria": ['SUV', 'MONOVOLUMEN', 'FAMILIAR'],
    "plazas_min": 5, # Familia
    "transmision_preferida": "automático", # Para comodidad familiar

    # Filtro económico (Modo 2 Contado)
    "modo": 2,
    "submodo": 1,
    "pago_contado": 30000.0,
    
    # Flags (simulando lo que calcularía finalizar_y_presentar_node)
    "penalizar_puertas_bajas": True, # Asumimos niños y uso frecuente
    "flag_penalizar_low_cost_comodidad": False, # Asumimos comodidad no es 10/10 para activar esto
    "flag_penalizar_deportividad_comodidad": False,
    "flag_penalizar_antiguo_por_tecnologia": True, # Si tecnología es importante y queremos coches más nuevos
    "aplicar_logica_distintivo_ambiental": True, # Si rating_impacto_ambiental fue >= 8
    "es_municipio_zbe": True # Simular que está en ZBE
}

# Pesos (ya normalizados, simulando la salida de normalize_weights)
# Estos deben sumar 1.0. Aquí un ejemplo donde impacto ambiental y sus derivados son altos.
pesos_test_familiar_eco = { 
    "estetica": 0.01, "premium": 0.01, "singular": 0.01, 
    "altura_libre_suelo": 0.03, "batalla": 0.03, "indice_altura_interior": 0.04,
    "ancho_general_score": 0.05, "traccion": 0.02, "reductoras": 0.0,
    "rating_fiabilidad_durabilidad": 0.07, "rating_seguridad": 0.08, 
    "rating_comodidad": 0.06,
    "rating_impacto_ambiental": 0.10, # Peso directo del rating
    "rating_tecnologia_conectividad": 0.05,
    "devaluacion": 0.04,
    "maletero_minimo_score": 0.05, "maletero_maximo_score": 0.05,
    "largo_vehiculo_score": 0.03,
    "autonomia_vehiculo": 0.05,
    "fav_bajo_peso": 0.06, # Activado por alto rating_impacto_ambiental
    "fav_bajo_consumo": 0.08, # Activado por alto rating_impacto_ambiental
    "fav_bajo_coste_uso_directo": 0.0, # Omitido por ahora
    "fav_bajo_coste_mantenimiento_directo": 0.0, # Omitido
    "par_motor_remolque_score": 0.0, "cap_remolque_cf_score": 0.0, "cap_remolque_sf_score": 0.0,
    "fav_menor_superficie_planta": 0.01, "fav_menor_diametro_giro": 0.01,
    "fav_menor_largo_garage": 0.01, "fav_menor_ancho_garage": 0.01, "fav_menor_alto_garage": 0.01,
    "deportividad_style_score": 0.0, "fav_menor_rel_peso_potencia_score": 0.0,
    "potencia_maxima_style_score": 0.0, "par_motor_style_score": 0.0, 
    "fav_menor_aceleracion_score": 0.0
}
# Asegurar que la suma de pesos sea aproximadamente 1.0 (ajusta los valores de arriba)
# print(f"Suma de pesos para test_familiar_eco: {sum(pesos_test_familiar_eco.values())}")

k_test = 5

print(f"\n--- Probando buscar_coches_bq con Escenario 1 ---")
print(f"Filtros para BQ: {json.dumps(filtros_test_familiar_eco, indent=2)}")
# print(f"Pesos para BQ: {json.dumps(pesos_test_familiar_eco, indent=2)}") # Descomentar para ver pesos

resultados_bq_directo = []
sql_ejecutada_directo = ""
params_ejecutados_directo = []

try:
    resultados_tupla = buscar_coches_bq(
        filtros=filtros_test_familiar_eco, 
        pesos=pesos_test_familiar_eco, 
        k=k_test
    )
    if isinstance(resultados_tupla, tuple) and len(resultados_tupla) == 3:
        resultados_bq_directo, sql_ejecutada_directo, params_ejecutados_directo = resultados_tupla
        print(f"\n--- SQL Ejecutada (Escenario 1) ---")
        print(sql_ejecutada_directo)
        # print(f"\n--- Parámetros SQL (Escenario 1) ---")
        # print(json.dumps(params_ejecutados_directo, indent=2))
    else:
        print("WARN: buscar_coches_bq no devolvió la tupla esperada.")
        resultados_bq_directo = resultados_tupla if isinstance(resultados_tupla, list) else []

except Exception as e:
    print(f"ERROR al ejecutar buscar_coches_bq directamente: {e}")
    traceback.print_exc()

print(f"\n--- Resultados Búsqueda Directa BQ ({len(resultados_bq_directo)} coches) ---")
if resultados_bq_directo:
    df_resultados_directo = pd.DataFrame(resultados_bq_directo)
    
    # Lista de columnas que te interesa ver (incluyendo las _scaled y el score)
    columnas_a_ver = [
        'nombre', 'marca', 'modelo', 'precio_compra_contado', 'score_total',
        'tipo_mecanica', 'tipo_carroceria', 'plazas', 'puertas', 
        'distintivo_ambiental', 'ocasion', 'anos_vehiculo',
        'fiabilidad', 'durabilidad', 'seguridad', 'comodidad', 'tecnologia', 
        'peso_original_kg', 'consumo_original', 
        # Algunas _scaled clave para este perfil
        'fiabilidad_scaled', 'durabilidad_scaled', 'seguridad_scaled', 'comodidad_scaled',
        'bajo_peso_scaled', 'bajo_consumo_scaled' 
    ]
    columnas_existentes = [col for col in columnas_a_ver if col in df_resultados_directo.columns]
    
    if 'score_total' in df_resultados_directo.columns: # Formatear score para mejor lectura
         df_resultados_directo['score_total'] = df_resultados_directo['score_total'].apply(lambda x: f"{x:.4f}" if isinstance(x, float) else x)
            
    if columnas_existentes:
        display(df_resultados_directo[columnas_existentes])
    else:
        print("No se encontraron columnas esperadas para mostrar. Mostrando DataFrame completo:")
        display(df_resultados_directo)
else:
    print("No se encontraron resultados para este escenario.")

# --- Puedes añadir más escenarios de prueba aquí ---
# print("\n--- ESCENARIO 2: ... ---")
# filtros_test_2 = { ... }
# pesos_test_2 = { ... }
# ... y repetir la llamada y visualización ...


## Prueba 2 Nodo finalizacion y busqueda coches

In [ ]:
# --- Celda de Prueba para Etapa Final del Agente en Jupyter Notebook ---
import pandas as pd
import logging
import traceback
import json 
from typing import Dict, List, Optional, Any
from langchain_core.messages import HumanMessage, BaseMessage,AIMessage

# --- 1. Importaciones Necesarias (Ajusta las rutas según tu proyecto) ---
# Clases de Estado y Enums
from graph.perfil.state import (
    EstadoAnalisisPerfil, PerfilUsuario, InfoPasajeros, 
    FiltrosInferidos, EconomiaUsuario, InfoClimaUsuario
)
from utils.enums import ( # Asume que están en utils/enums.py
    Transmision, NivelAventura, TipoUsoProfesional, 
    DimensionProblematica, EstiloConduccion, TipoMecanica
)

# Nodos del Grafo
from graph.perfil.nodes import (
    calcular_recomendacion_economia_modo1_node,
    obtener_tipos_carroceria_rag_node,
    calcular_flags_dinamicos_node,
    calcular_pesos_finales_node,
    formatear_tabla_resumen_node,
    buscar_coches_finales_node
)


# Utilidades (si son necesarias directamente en esta celda)

# from utils.rag_carroceria import get_recommended_carrocerias # Ya se llama dentro de un nodo
# from utils.weights import compute_raw_weights, normalize_weights # Ya se llaman dentro de un nodo
# from utils.formatters import formatear_preferencias_en_tabla # Ya se llama dentro de un nodo
# from utils.bigquery_tools import buscar_coches_bq # Ya se llama dentro de un nodo
# from utils.bq_logger import log_busqueda_a_bigquery # Ya se llama dentro de un nodo

# Configuración de LLMs (solo para que los nodos no fallen al importar)
# La llamada real al LLM de explicación ocurrirá en buscar_coches_finales_node
try:
    from config.llm import llm, llm_explicacion_coche # Asegúrate que llm_explicacion_coche esté definido
except ImportError:
    print("WARN: No se pudo importar la configuración de LLM. Las explicaciones podrían no funcionar.")
    llm_explicacion_coche = None


# Configurar logging básico para ver los DEBUG de los nodos
logging.basicConfig(level=logging.INFO) 
# Para ver más detalle:
# logging.getLogger("graph.perfil.nodes").setLevel(logging.DEBUG)
# logging.getLogger("utils.weights").setLevel(logging.DEBUG)
# logging.getLogger("utils.postprocessing").setLevel(logging.DEBUG)
# logging.getLogger("utils.rag_carroceria").setLevel(logging.DEBUG)
# logging.getLogger("utils.explanation_generator").setLevel(logging.DEBUG)


In [ ]:

# --- Configuración de Pandas para Mejor Visualización ---
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None) 
pd.set_option('display.width', 2000)    
pd.set_option('display.max_colwidth', 150) 

# --- 2. Simulación de un Estado Completo del Agente ---
# Define aquí el perfil completo del usuario, info de pasajeros, economía, etc.
# como si hubiera respondido a todas las preguntas.

# Ejemplo de Perfil de Usuario: "Familiar Práctico, con Garaje Ajustado y Énfasis en Seguridad"
test_codigo_postal = "17537" # Un CP para el que tengas datos climáticos
test_info_clima = InfoClimaUsuario( # Simula la salida de buscar_info_clima_node
    MUNICIPIO_ZBE=False, 
    ZONA_LLUVIAS=False, 
    ZONA_NIEBLAS=False, 
    ZONA_NIEVE=True, 
    ZONA_CLIMA_MONTA=True, 
    ZONA_GLP=False, 
    ZONA_GNV=False,
    cp_valido_encontrado=False,
    codigo_postal_consultado=test_codigo_postal
)

test_preferencias_usuario = PerfilUsuario(
    apasionado_motor="si", 
    valora_estetica="no", 
    coche_principal_hogar="sí", 
    uso_profesional="no", 
    tipo_uso_profesional=None, 
    prefiere_diseno_exclusivo="no", 
    altura_mayor_190="no", 
    peso_mayor_100="no", 
    transporta_carga_voluminosa="sí", 
    necesita_espacio_objetos_especiales="sí", # Para cochecitos, compra grande
    arrastra_remolque="no",
    tiene_garage="sí",
    problemas_aparcar_calle=None, # No aplica
    espacio_sobra_garage="no", # Garaje ajustado
    problema_dimension_garage=[DimensionProblematica.ANCHO, DimensionProblematica.LARGO], # Problemas de ancho y largo
    tiene_punto_carga_propio="sí", # Tiene punto de carga
    solo_electricos="no", # Abierto a híbridos
    prioriza_baja_depreciacion="sí",
    transmision_preferida=Transmision.AMBOS,
    estilo_conduccion=EstiloConduccion.TRANQUILO,
    aventura=NivelAventura.ocasional, # Viajes familiares
    rating_fiabilidad_durabilidad=9,
    rating_seguridad=5,
    rating_comodidad=8,
    rating_impacto_ambiental=7,
    rating_costes_uso=8, 
    rating_tecnologia_conectividad=10
)

test_info_pasajeros = InfoPasajeros( # Familia con 2 niños
    frecuencia="frecuente",
    num_ninos_silla=2,
    num_otros_pasajeros=1 # Total 2 niños + 1 adulto + conductor = 4-5 plazas
)

test_economia_usuario = EconomiaUsuario( # Modo 2 Contado
    modo=2, # Usar string como en el arreglo anterior
    submodo=1,
    pago_contado=25000.0,
    # El resto None para Modo 2 Contado
    ingresos=None, ahorro=None, cuota_max=None, entrada=None, anos_posesion=None
)

# Estado inicial simulado para la secuencia de finalización
# Los filtros inferidos iniciales pueden estar vacíos o con valores base del LLM de filtros
# ya que los nodos de post-procesamiento y RAG los llenarán.
initial_filtros_inferidos = FiltrosInferidos( 
    # estetica_min=1.0, # Ejemplo, estos se calcularán
    # premium_min=1.0,
    # singular_min=1.0,
    # tipo_mecanica=[TipoMecanica.GASOLINA] # Ejemplo
)

# Flags calculados por aplicar_filtros_pasajeros_node
# Para X=2, Z=1, frecuencia="frecuente":
# plazas_min = 2+1+1 = 4 (se pondrá en initial_filtros_inferidos por aplicar_filtros_pasajeros)
# priorizar_ancho = False (Z < 2)
# penalizar_puertas_bajas = True (frecuente y X >= 1)

# Simular el estado como estaría ANTES de la secuencia de finalización
# pero DESPUÉS de que aplicar_filtros_pasajeros_node haya corrido.
initial_state_dict: EstadoAnalisisPerfil = {
    "messages": [AIMessage(content="...")], # Un historial mínimo
    "codigo_postal_usuario": test_codigo_postal,
    "info_clima_usuario": test_info_clima,
    "preferencias_usuario": test_preferencias_usuario,
    "info_pasajeros": test_info_pasajeros,
    "economia": test_economia_usuario,
    "filtros_inferidos": initial_filtros_inferidos, # Se irá actualizando
    "pesos": None, # Se calculará
    "tabla_resumen_criterios": None, # Se generará
    "coches_recomendados": None, # Se buscarán
    "pregunta_pendiente": None,
    # Flags que vienen de aplicar_filtros_pasajeros_node
    "penalizar_puertas_bajas": True, # Basado en X=2, Z=1, freq="frecuente"
    "priorizar_ancho": False,        # Basado en Z=1
    # Flags que se calcularán en calcular_flags_dinamicos_node
    "flag_penalizar_low_cost_comodidad": None, 
    "flag_penalizar_deportividad_comodidad": None,
    "flag_penalizar_antiguo_por_tecnologia": None,
    "aplicar_logica_distintivo_ambiental": None,
    "es_municipio_zbe": None,
    # Campos temporales de CP (deberían estar None al llegar aquí)
    "codigo_postal_extraido_temporal": None,
    "tipo_mensaje_cp_llm": None,
    "_decision_cp_validation": None,
}

# --- 3. Ejecutar la Secuencia de Nodos de Finalización y Búsqueda ---
print("\n--- Iniciando Secuencia de Finalización y Búsqueda ---")
current_state = initial_state_dict.copy() # Trabajar con una copia

try:
    print("\n1. Ejecutando: calcular_recomendacion_economia_modo1_node")
    output_econ_modo1 = calcular_recomendacion_economia_modo1_node(current_state)
    current_state.update(output_econ_modo1)
    # print(f" Estado tras econ_modo1: {current_state.get('filtros_inferidos')}")

    print("\n2. Ejecutando: obtener_tipos_carroceria_rag_node")
    output_rag = obtener_tipos_carroceria_rag_node(current_state)
    current_state.update(output_rag)
    # print(f" Estado tras RAG (tipo_carroceria): {current_state.get('filtros_inferidos').tipo_carroceria}")

    print("\n3. Ejecutando: calcular_flags_dinamicos_node")
    output_flags = calcular_flags_dinamicos_node(current_state)
    current_state.update(output_flags)
    # print(f" Estado tras calc_flags: ZBE={current_state.get('es_municipio_zbe')}, ComodFlags={current_state.get('flag_penalizar_low_cost_comodidad')}")

    print("\n4. Ejecutando: calcular_pesos_finales_node")
    output_pesos = calcular_pesos_finales_node(current_state)
    current_state.update(output_pesos)
    # print(f" Estado tras calc_pesos: {current_state.get('pesos')}")

    print("\n5. Ejecutando: formatear_tabla_resumen_node")
    output_tabla = formatear_tabla_resumen_node(current_state)
    current_state.update(output_tabla)
    # print(f" Estado tras formatear_tabla (tabla_resumen_criterios): {current_state.get('tabla_resumen_criterios')}")
    
    # El nodo formatear_tabla_resumen ahora NO añade AIMessage, lo hace buscar_coches_finales_node
    # Así que no hay mensaje intermedio que mostrar aquí a menos que lo fuerces.

    print("\n6. Ejecutando: buscar_coches_finales_node")
    output_final_busqueda = buscar_coches_finales_node(current_state)
    current_state.update(output_final_busqueda)
    
    print("\n--- Mensaje Final del Agente (Combinado) ---")
    if current_state.get('messages'):
        # El último mensaje debería ser el combinado de tabla + coches
        current_state['messages'][-1].pretty_print()
    else:
        print("No se generó ningún mensaje final.")

except Exception as e_test:
    print(f"\nERROR durante la prueba de la secuencia de nodos: {e_test}")
    traceback.print_exc()

